In [1]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.tools.set_credentials_file(username='dniamir', api_key='fFtuut9mpQe6UAjqeMvH')
import datetime

mapbox_access_token = 'pk.eyJ1IjoiZG5pYW1pciIsImEiOiJjamR3NmV0eWsxb2NpMzJtdTB5c3ExN2FrIn0.MfxVkZvXoA-zZobcRf_25w'

init_notebook_mode(connected=True)
plotly.offline.init_notebook_mode()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
def Colours(This_Colour, Number_Of_Colours):
    
    ## Case for only 2 colours
    if(Number_Of_Colours == 2):
        Number_Of_Colours = 3
        if(This_Colour == 2):
            This_Colour = 3
    
    Interval2 = float(Number_Of_Colours)/float(6)
    Slope = float(1) / Interval2
    
    if(This_Colour == 0):
        R = 0
        G = 0
        B = 0
    elif(This_Colour <= 1*Interval2 + 1):
        R = 1
        G = Slope*(This_Colour - (0*Interval2 + 1))
        B = 0
        
    elif(This_Colour <= 2*Interval2 + 1):
        R = 1 - Slope*(This_Colour - (1*Interval2 + 1))
        G = 1
        B = 0
        
    elif(This_Colour <= 3*Interval2 + 1):
        R = 0
        G = 1
        B = Slope*(This_Colour - (2*Interval2 + 1))
        
    elif(This_Colour <= 4*Interval2 + 1):
        R = 0
        G = 1 - Slope*(This_Colour - (3*Interval2 + 1))
        B = 1
        
    elif(This_Colour <= 5*Interval2 + 1):
        R = Slope*(This_Colour - (4*Interval2 + 1))
        G = 0
        B = 1
        
    elif(This_Colour <= 6*Interval2 + 1):
        R = 1
        G = 0
        B = 1 - Slope*(This_Colour - (5*Interval2 + 1))

    if(R < 0):
        R = 0
    elif(R > 1):
        R = 1
    
    if(G < 0):
        G = 0
    elif(G > 1):
        G = 1
        
    if(B < 0):
        B = 0
    elif(B > 1):
        B = 1
    
    return [R*255, G*255, B*255]


In [277]:
URL_Dictionary = dict()
URL_Dictionary['Redwood City'] = 'https://www.zillow.com/homes/redwood-city_rb/'
URL_Dictionary['Menlo Park'] = 'https://www.zillow.com/homes/menlo-park_rb/'
URL_Dictionary['Palo Alto'] = 'https://www.zillow.com/homes/palo-alto_rb/'
URL_Dictionary['Mountain View'] = 'https://www.zillow.com/homes/Mountain-View-CA_rb/'
URL_Dictionary['Lost Altos'] = 'https://www.zillow.com/homes/Los-Altos-CA_rb/'

In [289]:
URL_Dictionary = dict()
URL_Dictionary['Redwood City'] = 'https://www.zillow.com/homes/for_sale/Redwood-City-CA/20128_rid/globalrelevanceex_sort/37.500908,-122.198997,37.443075,-122.279763_rect/13_zm/0_mmm/'
URL_Dictionary['Menlo Park'] = 'https://www.zillow.com/homes/for_sale/Menlo-Park-CA/39748_rid/globalrelevanceex_sort/37.517214,-122.098103,37.401528,-122.259636_rect/12_zm/0_mmm/'
URL_Dictionary['Palo Alto'] = 'https://www.zillow.com/homes/for_sale/Palo-Alto-CA/26374_rid/globalrelevanceex_sort/37.497469,-121.973992,37.265856,-122.297059_rect/11_zm/0_mmm/'
URL_Dictionary['Mountain View'] = 'https://www.zillow.com/homes/for_sale/Mountain-View-CA/32999_rid/globalrelevanceex_sort/37.471043,-122.000428,37.355285,-122.161961_rect/12_zm/0_mmm/'
URL_Dictionary['Los Altos'] = 'https://www.zillow.com/homes/for_sale/Los-Altos-CA/39511_rid/globalrelevanceex_sort/37.425116,-122.013989,37.309287,-122.175522_rect/12_zm/0_mmm/'
URL_Dictionary['San Francisco'] = 'https://www.zillow.com/homes/san-francisco_rb/'
URL_Dictionary['San Mateo'] = 'https://www.zillow.com/homes/san-mateo_rb/'
URL_Dictionary['Belmont'] = 'https://www.zillow.com/homes/belmont_rb/'
URL_Dictionary['Portola Valley'] = 'https://www.zillow.com/homes/Portola-Valley-CA_rb/'
URL_Dictionary['Pacifica'] = 'https://www.zillow.com/homes/Pacifica-CA_rb/'
URL_Dictionary['Cupertino'] = 'https://www.zillow.com/homes/Cupertino-CA_rb/'
URL_Dictionary['Sunnyvale'] = 'https://www.zillow.com/homes/Sunnyvale-CA_rb/'
URL_Dictionary['Santa Clara'] = 'https://www.zillow.com/homes/Santa-Clara-CA_rb/'

In [267]:
URL_Dictionary = dict()
URL_Dictionary['Everything'] = 'https://www.zillow.com/homes/for_sale/house,condo,apartment_duplex,mobile,townhouse_type/globalrelevanceex_sort/37.844766,-121.948586,37.38298,-122.594719_rect/10_zm/0_mmm/'

In [290]:
## Get Date
now = datetime.datetime.now()
df = pd.DataFrame(index=range(1,5))

## Input Parameters for Getting HTML Page
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/4.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

Cities = list(URL_Dictionary.keys())
Columns = ['Price', 'Area', 'Price Per Area', 'Beds', 'Baths', 'Street', 'City', 'State','Broker', 'Month', 'Day', 'Year']

## Loop over Cities
for City_Index, City_Name in enumerate(Cities):

    Page = 0
    
    ## Loop over Pages
#     while(True):
    for i in np.arange(1,4):
        
        Page = Page+1
        
        ## Define String for Page Number
        Page_Num = '/' + str(Page) + '_p'
        
        ## Getting City Names from the Dictionary
        url = URL_Dictionary[City_Name]
        url = url + Page_Num

        ## Check Site
        with requests.Session() as s:
            r = s.get(url, headers=req_headers)

        ## Scrape HTML Data
        soup = BeautifulSoup(r.content, 'lxml')
        
        ## Break Loop if the last page has been read
        Text = soup.get_text()
#         Test = soup.find('span', {'name': 'content'}).text
#         No_Results = Text.find('No matching results')
#         if(No_Results != -1):
#             break
#         break
        
        ## Read Specific Parts
#         price = soup.find('span', {'class': 'zsg-photo-card-price'}).text
#         info = soup.find('span', {'class': 'zsg-photo-card-info'}).text
#         address = soup.find('span', {'itemprop': 'address'}).text
        List_All = list()

        ## Iterate through Sections of the HTML
        Cards = soup.find_all('div', {'class': 'zsg-photo-card-caption'})
        for Index, li in enumerate(Cards):

            try:
                ## Get Raw Data
                Price = li.find('span', {'class': 'zsg-photo-card-price'}).text
                Address = li.find('span', {'class': 'zsg-photo-card-address'}).text
                Info = li.find('span', {'class': 'zsg-photo-card-info'}).text
                try: Broker = li.find('span', {'class': 'zsg-photo-card-broker-name'}).text
                except: Broker = 'NaN'

                ## Split up "Beds", "Baths", and "Square Feet"
                Info = Info.split()
                try: Beds = Info[0]
                except: Beds = 0
                    
                try: Baths = Info[3]
                except: Baths = 0
                
                try: 
                    Area = Info[6]
                    Area = Area.replace(',', '')                       # Remove Commas
                    Area = ''.join(ch for ch in Area if ch.isdigit())  # Remove Letters
                except: Area = 0

                Price = Price.replace(',', '')
                Price = Price.replace('$', '')
                Price = ''.join(ch for ch in Price if ch.isdigit())

                ## Split Address
                Address = Address.split(',')
                Street = Address[0]
                City = Address[1][1:]
                State = Address[2]
                State = State.replace(' ', '')

                ## Convert to Float
                Price = float(Price)
                Beds = float(Beds)
                Baths = float(Baths)
                Area = float(Area)
                PPA = Price / Area

                ## Date
                Year = now.year
                Month = now.month
                Day = now.day

                ## Append to List
                List_All.append([Price, Area, PPA, Beds, Baths, Street, City, State, Broker, Month, Day, Year])
                C = now.minute
                
            except:
                    B = now.minute
                    
        ## Combine all DataFrames
        if((City_Index == 0) and (Page == 1)):
            DF = pd.DataFrame(List_All, columns=Columns)
        else:
            DF2 = pd.DataFrame(List_All, columns=Columns)
            DF_Total = [DF, DF2]
            DF = pd.concat(DF_Total)
            DF = DF.drop_duplicates(subset=['Street', 'City', 'Month', 'Day', 'Year'])

        if(len(Cards) < 25):
            A = -5
            break
        

In [293]:
## Read last Dataframe and Append
Data_File = 'Data.csv'
Directory = './Data/' + Data_File
DF_Old = pd.read_csv(Directory, sep=',')

## Remove First Column (Indices)
Columns_Old = list(DF_Old)
DF_Old = DF_Old.drop(DF_Old.columns[0], axis=1)
DF_Old = DF_Old[Columns]

In [319]:
DF_Total = [DF_Old, DF]
DF = pd.concat(DF_Total)
        
## Revmoe Duplicate Rows
DF = DF.drop_duplicates(subset=['Street', 'City', 'Month', 'Day', 'Year'])

## Write Dataframe to a File
DF.to_csv(Directory, sep=',')

In [306]:
## Filter Data Frames
DF = DF[DF['Price'] > 100000]

In [309]:
DF[DF['Street'] == '3513 Waverley St']

,Price,Area,Price Per Area,Beds,Baths,Street,City,State,Broker,Month,Day,Year
128,5698000.0,5292.0,1076.719577,5.0,5.0,3513 Waverley St,Palo Alto,CA,Pacific Union International Inc.,5,14,2018
236,4998000.0,5292.0,944.444444,5.0,5.0,3513 Waverley St,Palo Alto,CA,Pacific Union International Inc.,6,5,2018
16,4998000.0,5292.0,944.444444,5.0,5.0,3513 Waverley St,Palo Alto,CA,Pacific Union International Inc.,6,6,2018


In [318]:
## Price Changing Over Time

Streets = np.unique(DF['Street'].values)
Cities = np.unique(DF['City'].values)

for Street_Index, Street in enumerate(Streets):

    Colour = Colours(Street_Index, len(Streets))
    Colour = 'rgb(' + str(Colour[0]) + ',' + str(Colour[1]) + ',' + str(Colour[2]) + ')'

    DF2 = DF[DF['Street'] == Street]
    
    X = DF2['Month'].values
    Y = DF2['Price'].values
    Y = Y - Y[0]

    ## Create  Trace
    trace1 = go.Scatter(
        y = Y,
        x = X,
        mode = 'lines+markers',
        name = DF2['City'].values[0],
        text = DF2['Street'].values,
        marker = dict(color = Colour, size = 10, line = dict(color = 'black', width = 2)),
        opacity = 0.5,
        showlegend=False
    )

    if(Street_Index == 0): data = [trace1, trace2]
    else: data = data + [trace1, trace2]


layout = dict(title = 'Price over Time',
              xaxis = dict(title='Month', zeroline = False),
              yaxis = dict(title='Price [$]', zeroline = False)
             )

fig = dict(data=data, layout=layout)
plotly.offline.plot(fig)

'file:///Users/Darien/Python/Real Estate Scraping/temp-plot.html'

In [ ]:
## Price Vs. Square FT - Plotly

Cities = np.unique(DF['City'].values)

for City_Index, City in enumerate(Cities):

    Colour = Colours(City_Index, len(Cities))
    Colour = 'rgb(' + str(Colour[0]) + ',' + str(Colour[1]) + ',' + str(Colour[2]) + ')'

    DF2 = DF[DF['City'] == City]
    X = DF2['Area'].values
    Y = DF2['Price'].values

    ## Create  Trace
    trace1 = go.Scatter(
        y = Y,
        x = X,
        mode = 'markers',
        name = DF2['City'].values[0],
        text = DF2['Street'].values,
        marker = dict(color = Colour, size = 10, line = dict(color = 'black', width = 2)),
        line = dict(dash = 'dot'),
        opacity = 0.5
    )

    ## Create  Line of Best Fit
    Fit = np.polyfit(X, Y, 1)
    Y_Fit = np.polyval(Fit, X)

    trace2 = go.Scatter(
        y = [np.min(Y_Fit), np.max(Y_Fit)],
        x = [np.min(X), np.max(X)],
        name = DF2['City'].values[0],
        text = DF2['Street'].values,
        mode = 'lines',
        line = dict(color = Colour, dash='dash'),
        showlegend=False,
    )
    
    if(City_Index == 0): data = [trace1, trace2]
    else: data = data + [trace1, trace2]


layout = dict(title = 'Price per Sq FT',
              xaxis = dict(title='Area [Sq FT]', zeroline = False),
              yaxis = dict(title='Price [$]', zeroline = False)
             )

fig = dict(data=data, layout=layout)
plotly.offline.plot(fig)